In [ ]:
#!/usr/bin/env python3
import os
import json
import h5py
import argparse
import pandas as pd
import numpy as np
import tinydb as db
from tinydb.storages import MemoryStorage
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib.colors import LogNorm

import boost_histogram as bh
import pickle as pl

import scipy.stats as stats
import scipy.optimize as opt
from pygama import DataGroup
import pygama.lh5 as lh5
import pygama.analysis.histograms as pgh
import pygama.analysis.peak_fitting as pgf
import cage_utils
plt.rcParams['figure.figsize'] = [14, 10]

In [ ]:
dg = DataGroup('../processing/cage.json', load=True)
dsp_dir = '/global/cfs/cdirs/legend/data/cage/LH5/dsp'

In [ ]:
run = 255
df = dg.fileDB.query(f'run == {run} and skip==False')

raw_dir = '/global/cfs/cdirs/legend/data/cage/LH5/dsp'
f_raw = f"{raw_dir}/{df['dsp_file'].iloc[-2]}"
raw = h5py.File(f_raw)

print(raw['ORSIS3302DecoderForEnergy']['dsp'].keys())


In [ ]:
hit_dir = '/global/cfs/cdirs/legend/data/cage/LH5/hit'
f_hit = f"{hit_dir}/{df['hit_file'].iloc[0]}"
hit = h5py.File(f_hit)

print(hit['ORSIS3302DecoderForEnergy']['hit'].keys())


In [ ]:
trapEftp_cal = np.empty(0)
print(len(df))
for i in range(len(df)):
    f_hit = f"{hit_dir}/{df['hit_file'].iloc[i]}"
    hit = h5py.File(f_hit)
    trapEftp_cal = np.append(trapEftp_cal, np.array(hit['ORSIS3302DecoderForEnergy']['hit']['trapEftp_cal']))

print(trapEftp_cal[0])

In [ ]:
#trapEftp_cal = trapEftp_cal[np.logical_not(np.isnan(trapEftp_cal))]
trapEftp_cal = trapEftp_cal[trapEftp_cal>0]

In [ ]:
print(np.amin(trapEftp_cal))
print(np.amax(trapEftp_cal))

In [ ]:
fig = plt.hist(trapEftp_cal, bins=10000)
plt.yscale('log')


In [ ]:
run = 245
df = dg.fileDB.query(f'run=={run} and skip==False')

bins = np.arange(0,40000,4)
dsp_dir = '/global/cfs/cdirs/legend/data/cage/LH5/dsp'

alp_runtime = 0
trapEftp = np.empty(0)

for i in range(len(df)):
    f_dsp = f"{dsp_dir}/{df['dsp_file'].iloc[i]}"
    dsp = h5py.File(f_dsp)
    trapEftp = np.append(trapEftp, np.array(dsp['ORSIS3302DecoderForEnergy']['dsp']['trapEftp']))
    alp_runtime += df['runtime'].iloc[i]

In [ ]:
alp_hist = plt.hist(trapEftp,bins=bins)
plt.yscale('log')
plt.ylabel('counts/sec')
plt.xlim(1500,3000)

In [ ]:
print(np.amin(trapEftp))
print(np.amax(trapEftp))

In [ ]:
alp_hist = plt.hist(trapEftp,bins=bins)
plt.yscale('log')
plt.ylabel('counts/sec')
plt.xlim(2000,4000)

In [ ]:
run = 256
df = dg.fileDB.query(f'run == {run} and skip==False')

bkg_runtime = 0


trapEftp_bkg = np.empty(0)

for i in range(len(df)):
    f_dsp = f"{dsp_dir}/{df['dsp_file'].iloc[i]}"
    dsp = h5py.File(f_dsp)
    trapEftp_bkg = np.append(trapEftp_bkg, np.array(dsp['ORSIS3302DecoderForEnergy']['dsp']['trapEftp']))
    bkg_runtime += df['runtime'].iloc[i]

print(trapEftp_bkg[0])
print(bkg_runtime)

In [ ]:
bkg_hist = plt.hist(trapEftp_bkg, bins=bins)
plt.yscale('log')
plt.ylabel('counts/sec')

In [ ]:
bkg_hist = plt.hist(trapEftp_bkg, bins=bins)
plt.yscale('log')
plt.ylabel('counts/sec')
plt.xlim(2000,4000)

In [ ]:
bkg_sub = alp_hist[0]/(33*1800) - bkg_hist[0]/(25*1800)

In [ ]:
sub_hist = plt.hist(bins[:-1], bins=bins, weights=bkg_sub)
plt.yscale('log')

In [ ]:
dsp_dir = '/global/cfs/cdirs/legend/data/cage/LH5/dsp'

cal_const = {
    245: (1460, ),
    246: (1460, ),
    247: (1460, ),
    248: (1460, ),
    249: (1460, ),
    250: (1460, ),
    251: (1460, ),
    252: (1460, ),
    253: (1460, ),
    254: (1460, ),
    255: (1460, ),
    256: (1460, ),
    257: (1460, ),
    258: (1460, ),
    259: (1460, ),
    260: (1460, ),
    261: (1460, ),
    262: (1460, )
}
    

#runtimes are given in minutes
def bkg_subtract(alp_run, bkg_run, alp_time=0, bkg_time=0, plot=True):
    alp_df = dg.fileDB.query(f'run == {alp_run} and skip==False')
    bkg_df = dg.fileDB.query(f'run == {bkg_run} and skip==False')
    
    alp_runtime = 0
    trapEftp_alp = np.empty(0)
    for i in range(len(alp_df)):
        f_dsp = f"{dsp_dir}/{alp_df['dsp_file'].iloc[i]}"
        dsp = h5py.File(f_dsp)
        trapEftp_alp = np.append(trapEftp_alp, np.array(dsp['ORSIS3302DecoderForEnergy']['dsp']['trapEftp']))
        alp_runtime += alp_df['runtime'].iloc[i]
    
    bkg_runtime = 0
    trapEftp_bkg = np.empty(0)
    
    for i in range(len(bkg_df)):
        f_dsp = f"{dsp_dir}/{bkg_df['dsp_file'].iloc[i]}"
        dsp = h5py.File(f_dsp)
        trapEftp_bkg = np.append(trapEftp_bkg, np.array(dsp['ORSIS3302DecoderForEnergy']['dsp']['trapEftp']))
        bkg_runtime += bkg_df['runtime'].iloc[i]

    if np.isnan(bkg_runtime):
        bkg_runtime = bkg_time
    if np.isnan(alp_runtime):
        alp_runtime = alp_time
        
    print(alp_runtime, bkg_runtime)    
    
    bins = np.arange(0,40000,4)
    alp_hist = np.histogram(trapEftp_alp, bins=bins)
    bkg_hist = np.histogram(trapEftp_bkg, bins=bins)

    bkg_sub = alp_hist[0]/(alp_runtime*60) - bkg_hist[0]/(bkg_runtime*60)
    
    if plot:
        plt.hist(bins[:-1], bins=bins, weights=bkg_sub)
        plt.ylabel('alpha rate [count/sec]')
        plt.xlabel('linear [mm]')
        plt.yscale('log')
        plt.show()
    
    return bkg_sub

In [ ]:
bkg_subtract(244, 250, 33*30, 25*30)

In [ ]:
rot0_lin5 = bkg_subtract(245, 250, 4*30, 25*30)


In [ ]:
print(np.sum(rot0_lin5[1250:2500]))

In [ ]:
rot0_lin6 = bkg_subtract(246, 250, 4*30, 25*30)


In [ ]:
print(np.sum(rot0_lin6[1250:2500]))

In [ ]:
rot0_lin7 = bkg_subtract(247, 250, 4*30, 25*30)
print(np.sum(rot0_lin7[1250:2500]))

In [ ]:
rot0_lin8 = bkg_subtract(248, 250, 4*30, 25*30)
print(np.sum(rot0_lin8[1250:2500]))

In [ ]:
rot0_lin9 = bkg_subtract(249, 250, 4*30, 25*30)
print(np.sum(rot0_lin9[1250:2500]))

In [ ]:
rot0_lin10 = bkg_subtract(240, 241, 4*30, 25*30)
print(np.sum(rot0_lin10[1250:2500]))

In [ ]:
print(np.sum(rot180_lin9))

In [ ]:
rot0_lin5 = bkg_subtract(245, 250, 4*30, 24.5*30, False)
rot0_lin6 = bkg_subtract(246, 250, 4*30, 24.5*30, False)
rot0_lin7 = bkg_subtract(247, 250, 4*30, 24.5*30, False)
rot0_lin8 = bkg_subtract(248, 250, 4*30, 24.5*30, False)
rot0_lin9 = bkg_subtract(249, 250, 4*30, 24.5*30, False)
rot0_lin10 = bkg_subtract(240, 241, 9*30, 20*30, False)

In [ ]:
print(np.sum(rot180_lin6))

In [ ]:
rot180_lin5 = bkg_subtract(251, 256, 3*30, 30*30, False)
rot180_lin6 = bkg_subtract(252, 256, 3*30, 30*30, False)
rot180_lin7 = bkg_subtract(253, 256, 3*30, 30*30, False)
rot180_lin8 = bkg_subtract(254, 256, 3*30, 30*30, False)
rot180_lin9 = bkg_subtract(255, 256, 3*30, 30*30, False)

In [ ]:
rot145_lin5 = bkg_subtract(257, 263, 3*30, 23.7*30, False)
rot145_lin6 = bkg_subtract(258, 263, 3*30, 23.7*30, False)
rot145_lin7 = bkg_subtract(259, 263, 3*30, 23.7*30, False)
rot145_lin8 = bkg_subtract(260, 263, 3*30, 23.7*30, False)
rot145_lin9 = bkg_subtract(261, 263, 3*30, 23.7*30, False)
rot145_lin10 = bkg_subtract(262, 263, 3*30, 23.7*30, False)

In [ ]:
rot0 = [np.sum(rot0_lin5[1250:2500]), np.sum(rot0_lin6[1250:2500]), np.sum(rot0_lin7[1250:2500]), np.sum(rot0_lin8[1250:2500]), np.sum(rot0_lin9[1250:2500]), np.sum(rot0_lin10[1250:2500])]
rot180 = [np.sum(rot180_lin5[1250:2500]), np.sum(rot180_lin6[1250:2500]), np.sum(rot180_lin7[1250:2500]), np.sum(rot180_lin8[1250:2500]), np.sum(rot180_lin9[1250:2500])]
rot145 = [np.sum(rot145_lin5[1250:2500]), np.sum(rot145_lin6[1250:2500]), np.sum(rot145_lin7[1250:2500]), np.sum(rot145_lin8[1250:2500]), np.sum(rot145_lin9[1250:2500]), np.sum(rot145_lin10[1250:2500])]
lin = [5, 6, 7, 8, 9]
lin2 = [5, 6, 7, 8, 9, 10]

plt.figure(facecolor='w')
plt.scatter(lin2, rot0, label='rotary 0')
plt.scatter(lin, rot180, label='rotary 180')
plt.scatter(lin2, rot145, label='rotary 145')
plt.legend()

#plt.axvline(7.2, color='blue')
#plt.axvline(6.7, color='orange')
#plt.axvline(6.7, color='green')

plt.xlabel('linear [mm]')
plt.ylabel('rate (count/sec)')

In [ ]:
rot180_lin9 = bkg_subtract(255, 256, 3*30, 30*30, True)

In [ ]:
np.sum(rot180_lin9)

In [ ]:
rot0_lin10 = bkg_subtract(240, 241, 8*30, 20*30, True)
print(np.sum(rot0_lin10))